In [ ]:
# | default_exp _components.producer_decorator

In [ ]:
# | export

import functools
import json
from asyncio import iscoroutinefunction  # do not use the version from inspect
from collections import namedtuple
from dataclasses import dataclass
from typing import *

from aiokafka import AIOKafkaProducer
from pydantic import BaseModel

In [ ]:
import asyncio
from contextlib import asynccontextmanager
from unittest.mock import Mock

from pydantic import Field

from fastkafka._components.aiokafka_producer_manager import AIOKafkaProducerManager
from fastkafka._testing.local_broker import LocalKafkaBroker
from fastkafka._testing.test_utils import mock_AIOKafkaProducer_send

In [ ]:
# | export


BaseSubmodel = TypeVar("BaseSubmodel", bound=BaseModel)
BaseSubmodel


@dataclass
class KafkaEvent(Generic[BaseSubmodel]):
    """
    A generic class for representing Kafka events. Based on BaseSubmodel, bound to pydantic.BaseModel

    Attributes:
        message (BaseSubmodel): The message contained in the Kafka event, can be of type pydantic.BaseModel.
        key (bytes, optional): The optional key used to identify the Kafka event.
    """

    message: BaseSubmodel
    key: Optional[bytes] = None


KafkaEvent.__module__ = "fastkafka"

In [ ]:
event = KafkaEvent("Some message")
assert event.message == "Some message"
assert event.key == None

event = KafkaEvent("Some message", b"123")
assert event.message == "Some message"
assert event.key == b"123"

In [ ]:
# | export

ProduceReturnTypes = Union[BaseModel, KafkaEvent[BaseModel]]

ProduceCallable = Union[
    Callable[..., ProduceReturnTypes], Callable[..., Awaitable[ProduceReturnTypes]]
]

In [ ]:
# | export


def _to_json_utf8(o: Any) -> bytes:
    """Converts to JSON and then encodes with UTF-8"""
    if hasattr(o, "json"):
        return o.json().encode("utf-8")  # type: ignore
    else:
        return json.dumps(o).encode("utf-8")

In [ ]:
assert _to_json_utf8({"a": 1, "b": [2, 3]}) == b'{"a": 1, "b": [2, 3]}'


class A(BaseModel):
    name: str = Field()
    age: int


assert _to_json_utf8(A(name="Davor", age=12)) == b'{"name": "Davor", "age": 12}'

In [ ]:
# | export


def _wrap_in_event(message: Union[BaseModel, KafkaEvent]) -> KafkaEvent:
    return message if type(message) == KafkaEvent else KafkaEvent(message)

In [ ]:
message = A(name="Davor", age=12)
wrapped = _wrap_in_event(message)

assert type(wrapped) == KafkaEvent
assert wrapped.message == message
assert wrapped.key == None

In [ ]:
message = KafkaEvent(A(name="Davor", age=12), b"123")
wrapped = _wrap_in_event(message)

assert type(wrapped) == KafkaEvent
assert wrapped.message == message.message
assert wrapped.key == b"123"

In [ ]:
# | export


def producer_decorator(
    producer_store: Dict[str, Any], func: ProduceCallable, topic: str
) -> ProduceCallable:
    """todo: write documentation"""

    @functools.wraps(func)
    async def _produce_async(
        *args: List[Any],
        producer_store: Dict[str, Any] = producer_store,
        f: Callable[..., Awaitable[ProduceReturnTypes]] = func,  # type: ignore
        **kwargs: Any
    ) -> ProduceReturnTypes:
        return_val = await f(*args, **kwargs)
        wrapped_val = _wrap_in_event(return_val)
        _, producer, _ = producer_store[topic]
        fut = await producer.send(
            topic, _to_json_utf8(wrapped_val.message), key=wrapped_val.key
        )
        msg = await fut
        return return_val

    @functools.wraps(func)
    def _produce_sync(
        *args: List[Any],
        producer_store: Dict[str, Any] = producer_store,
        f: Callable[..., ProduceReturnTypes] = func,  # type: ignore
        **kwargs: Any
    ) -> ProduceReturnTypes:
        return_val = f(*args, **kwargs)
        wrapped_val = _wrap_in_event(return_val)
        _, producer, _ = producer_store[topic]
        producer.send(topic, _to_json_utf8(wrapped_val.message), key=wrapped_val.key)
        return return_val

    return _produce_async if iscoroutinefunction(func) else _produce_sync

In [ ]:
class MockMsg(BaseModel):
    name: str = "Micky Mouse"
    id: int = 123


mock_msg = MockMsg()

topic = "test_topic"

In [ ]:
@asynccontextmanager
async def mock_producer_env(
    is_sync: bool,
) -> AsyncGenerator[
    Tuple[Mock, Union[AIOKafkaProducer, AIOKafkaProducerManager]], None
]:
    try:
        with mock_AIOKafkaProducer_send() as send_mock:
            async with LocalKafkaBroker(topics=[topic]) as bootstrap_server:
                producer = AIOKafkaProducer(bootstrap_servers=bootstrap_server)
                if is_sync:
                    producer = AIOKafkaProducerManager(producer)
                await producer.start()
                yield send_mock, producer
    finally:
        await producer.stop()

In [ ]:
async def func(mock_msg: MockMsg) -> MockMsg:
    return mock_msg


async with mock_producer_env(is_sync=False) as (send_mock, producer):
    test_func = producer_decorator({topic: (None, producer, None)}, func, topic)

    assert iscoroutinefunction(test_func) == True

    value = await test_func(mock_msg)

    send_mock.assert_called_once_with(topic, mock_msg.json().encode("utf-8"), key=None)

    assert value == mock_msg

[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
[INFO] fastkafka._testing.local_broker: Starting zookeeper...
[INFO] fastkafka._testing.local_broker: Starting kafka...
[INFO] fastkafka._testing.local_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 561196...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 561196 terminated.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 560836...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 560836 terminated.


In [ ]:
def func(mock_msg: MockMsg) -> MockMsg:
    return mock_msg


async with mock_producer_env(is_sync=True) as (send_mock, producer):
    test_func = producer_decorator({topic: (None, producer, None)}, func, topic)

    assert iscoroutinefunction(test_func) == False

    value = test_func(mock_msg)
    await asyncio.sleep(1)

    send_mock.assert_called_once_with(topic, mock_msg.json().encode("utf-8"), key=None)

    assert value == mock_msg

[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.local_broker: Starting zookeeper...
[INFO] fastkafka._testing.local_broker: Starting kafka...
[INFO] fastkafka._testing.local_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._components.aiokafka_producer_manager: AIOKafkaProducerManager.start(): Entering...
[INFO] fastkafka._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Starting...
[INFO] fastkafka._components.aiokafka_producer_manager: _aiokafka_producer_manager(): Starting send_stream
[INFO] fastkafka._components.aiokafka_producer_manager: AIOKafkaProducerManager.start(): Finished.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 562340...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 562340 terminated.
[INFO] fastkafka._components._subproc

In [ ]:
test_key = b"some_key"

In [ ]:
async def func(mock_msg: MockMsg) -> KafkaEvent[MockMsg]:
    return KafkaEvent(mock_msg, key=test_key)


async with mock_producer_env(is_sync=False) as (send_mock, producer):
    test_func = producer_decorator({topic: (None, producer, None)}, func, topic)

    assert iscoroutinefunction(test_func) == True

    value = await test_func(mock_msg)

    send_mock.assert_called_once_with(
        topic, mock_msg.json().encode("utf-8"), key=test_key
    )

    assert value == KafkaEvent(mock_msg, key=test_key)

[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.local_broker: Starting zookeeper...
[INFO] fastkafka._testing.local_broker: Starting kafka...
[INFO] fastkafka._testing.local_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 563478...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 563478 terminated.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 563117...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 563117 terminated.


In [ ]:
async def func(mock_msg: MockMsg) -> KafkaEvent[MockMsg]:
    return KafkaEvent(mock_msg, key=test_key)


async with mock_producer_env(is_sync=False) as (send_mock, producer):
    test_func = producer_decorator({topic: (None, producer, None)}, func, topic)

    assert iscoroutinefunction(test_func) == True

    value = await test_func(mock_msg)

    send_mock.assert_called_once_with(
        topic, mock_msg.json().encode("utf-8"), key=test_key
    )

    assert value == KafkaEvent(mock_msg, key=test_key)

[INFO] fastkafka._components.test_dependencies: Java is already installed.
[INFO] fastkafka._components.test_dependencies: Kafka is installed.
[INFO] fastkafka._testing.local_broker: Starting zookeeper...
[INFO] fastkafka._testing.local_broker: Starting kafka...
[INFO] fastkafka._testing.local_broker: Local Kafka broker up and running on 127.0.0.1:9092
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 564618...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 564618 terminated.
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 564257...
[INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 564257 terminated.
